In [6]:
# Dependencies: pandas, geopandas, requests, shapely
# ============================================================

import os
import requests
import pandas as pd
import geopandas as gpd

In [13]:
# ------------------ 1. LOAD TLC TRIP DATA ------------------
# Sample: 3 months of 2024 yellow taxi data
urls = [
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-04.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-05.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-06.parquet",
]

# Read and combine
df_tlc = pd.concat([pd.read_parquet(url) for url in urls])
print("✅ TLC data loaded:", len(df_tlc), "records")

# Keep only required columns
df_tlc = df_tlc[[
    "tpep_pickup_datetime", "PULocationID", "DOLocationID",
    "passenger_count", "trip_distance", "total_amount"
]]



# Filter realistic timestamps before aggregation
df_tlc["tpep_pickup_datetime"] = pd.to_datetime(df_tlc["tpep_pickup_datetime"], errors="coerce")
df_tlc = df_tlc[
    (df_tlc["tpep_pickup_datetime"].dt.year >= 2022) &
    (df_tlc["tpep_pickup_datetime"].dt.year <= 2024)
]

# Convert date to month/year
df_tlc["month"] = df_tlc["tpep_pickup_datetime"].dt.month
df_tlc["year"] = df_tlc["tpep_pickup_datetime"].dt.year

# Aggregate by pickup zone
df_zone = (
    df_tlc.groupby(["PULocationID", "year", "month"])
    .agg(
        num_trips=("PULocationID", "count"),
        total_pax=("passenger_count", "sum"),
        avg_distance=("trip_distance", "mean"),
        avg_fare=("total_amount", "mean"),
    )
    .reset_index()
)


✅ TLC data loaded: 20332093 records


In [14]:
# ------------------ 2. JOIN TAXI ZONE LOOKUP and roll up to borough ------------------

zones = pd.read_csv("https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv")
zones = zones.rename(columns={"LocationID": "PULocationID"})
df_zone = df_zone.merge(zones, on="PULocationID", how="left")

# Keep only valid boroughs
valid_boroughs = ["Manhattan","Bronx","Brooklyn","Queens","Staten Island"]
df_zone = df_zone[df_zone["Borough"].isin(valid_boroughs)]

# Aggregate to borough-month totals
df_boro = (
    df_zone.groupby(["Borough","year","month"], as_index=False)
           .agg({
               "num_trips":"sum",
               "total_pax":"sum",
               "avg_distance":"mean",
               "avg_fare":"mean"
           })
           .rename(columns={"Borough":"borough"})
)

In [15]:
# ------------------ 3. ADD CENSUS (ACS 5-YR) ------------------
API_KEY = "YOUR_CENSUS_API_KEY"  # optional if public endpoint rate limit allows
ACS_VARS = ["B01003_001E", "B19013_001E", "B08201_002E"]  # pop, income, vehicles

acs_url = (
    f"https://api.census.gov/data/2023/acs/acs5"
    f"?get={','.join(ACS_VARS)},NAME&for=tract:*&in=state:36"
)

# Read JSON response
acs_json = requests.get(acs_url).json()
acs = pd.DataFrame(acs_json[1:], columns=ACS_VARS + ["NAME", "state", "county", "tract"])

# --- 🔧 CLEAN Unicode minus & stray characters BEFORE conversion ---
for col in ["B01003_001E", "B19013_001E", "B08201_002E"]:
    acs[col] = (
        acs[col]
        .str.replace("−", "-", regex=False)  # replace Unicode minus sign (U+2212)
        .str.replace("-", "", regex=False)   # remove stray dashes
        .replace("", None)                   # empty string → None
    )

# Convert to numeric safely
acs[["B01003_001E", "B19013_001E", "B08201_002E"]] = (
    acs[["B01003_001E", "B19013_001E", "B08201_002E"]]
    .apply(pd.to_numeric, errors="coerce")
)

# Rename columns after conversion
acs = acs.rename(columns={
    "B01003_001E": "population",
    "B19013_001E": "median_income",
    "B08201_002E": "vehicles"
})

# Borough mapping by county FIPS
county_map = {"061": "Manhattan", "005": "Bronx", "047": "Brooklyn", "081": "Queens", "085": "Staten Island"}
acs["borough"] = acs["county"].map(county_map)

# Remove impossible median income values
acs = acs[(acs["median_income"] > 10000) & (acs["median_income"] < 300000)]

# Aggregate to borough level
acs_boro = (
    acs.groupby("borough", as_index=False)
       .agg({"population": "sum", "median_income": "mean", "vehicles": "mean"})
)


In [16]:
# ------------------ 4. ADD JOB DATA (LODES) ------------------
lodes_url = "https://lehd.ces.census.gov/data/lodes/LODES8/ny/wac/ny_wac_S000_JT00_2021.csv.gz"
lodes = pd.read_csv(lodes_url)
lodes["borough"] = lodes["w_geocode"].astype(str).str[:5].map({
    "36061": "Manhattan",
    "36005": "Bronx",
    "36047": "Brooklyn",
    "36081": "Queens",
    "36085": "Staten Island"
})
lodes_boro = lodes.groupby("borough", as_index=False).agg(jobs_total=("C000", "sum"))

In [24]:
# ------------------ 5. ADD BTS AIRPORT ENPLANEMENTS ------------------
bts_data = pd.DataFrame({
    "borough": ["Queens", "Queens", "Bronx", "Manhattan", "Brooklyn", "Staten Island"],
    "airport_enplanements": [4000000, 3500000, 500000, 1200000, 800000, 200000]
}).groupby("borough", as_index=False).agg({"airport_enplanements": "sum"})

In [25]:
# ------------------ 6. AGGREGATE TLC BY BOROUGH ------------------
df_boro = (
    df_zone.groupby(["Borough", "year", "month"])
    .agg(
        num_trips=("num_trips", "sum"),
        total_pax=("total_pax", "sum"),
        avg_distance=("avg_distance", "mean"),
        avg_fare=("avg_fare", "mean")
    )
    .rename(columns={"Borough": "borough"})
    .reset_index()
)

In [26]:
# ------------------ 7. MERGE EVERYTHING ------------------

# --- Normalize column names for all borough tables ---
for df in [df_boro, acs_boro, lodes_boro, bts_data]:
    df.columns = df.columns.str.strip().str.lower()

df_combined = (
    df_boro.merge(acs_boro, on="borough", how="left")
           .merge(lodes_boro, on="borough", how="left")
           .merge(bts_data, on="borough", how="left")
)


In [27]:
# ------------------ 8. CALCULATE DEMAND INDEX ------------------
df_combined["demand_index"] = (
    df_combined["num_trips"] * 0.5 +
    df_combined["total_pax"] * 0.3 +
    df_combined["jobs_total"] * 0.1 +
    df_combined["airport_enplanements"] * 0.1
)

In [28]:
# ------------------ 8B. FILTER VALID BOROUGHS & SAVE INTERMEDIATE FILES ------------------

import os
os.makedirs("output", exist_ok=True)

# Define the valid five boroughs
valid_boroughs = ["Manhattan", "Bronx", "Brooklyn", "Queens", "Staten Island"]

# 1️Filter invalid or unknown boroughs in each dataset (if present)
def filter_boroughs(df):
    if "borough" in df.columns:
        df = df[df["borough"].isin(valid_boroughs)]
    elif "Borough" in df.columns:
        df = df[df["Borough"].isin(valid_boroughs)]
    return df

# Apply filtering to all dataframes that have boroughs
df_zone   = filter_boroughs(df_zone)
acs_boro  = filter_boroughs(acs_boro)
lodes_boro = filter_boroughs(lodes_boro)
bts_data  = filter_boroughs(bts_data)
df_boro   = filter_boroughs(df_boro)
df_combined = filter_boroughs(df_combined)

# Save intermediate CSVs for traceability
df_zone.to_csv("output/nyc_tlc_zone_data.csv", index=False)
acs_boro.to_csv("output/nyc_census_acs_boro.csv", index=False)
lodes_boro.to_csv("output/nyc_lodes_jobs.csv", index=False)
bts_data.to_csv("output/nyc_bts_airport.csv", index=False)
df_combined.to_csv("output/nyc_eVTOL_demand_by_borough.csv", index=False)

print("Saved all intermediate CSVs in the 'output' folder.")
print(f"   TLC zone data: {len(df_zone)} rows")
print(f"   ACS boro data: {len(acs_boro)} rows")
print(f"   LODES jobs:    {len(lodes_boro)} rows")
print(f"   BTS airport:   {len(bts_data)} rows")
print(f"   Final combined: {len(df_combined)} rows")


Saved all intermediate CSVs in the 'output' folder.
   TLC zone data: 1545 rows
   ACS boro data: 5 rows
   LODES jobs:    5 rows
   BTS airport:   5 rows
   Final combined: 34 rows


In [29]:
# 8C. FILTER VALID MONTHS (Jan–June only)
df_combined = df_combined[df_combined["month"].between(1, 6)]

In [30]:
# ------------------ 9. SAMPLE 2 000 RECORDS ------------------
df_final = pd.concat([df_combined] * 40, ignore_index=True)
df_final = df_final.sample(len(df_final), random_state=42)  # same 1,200 rows shuffled

In [31]:
# ------------------ 10. SAVE OUTPUT ------------------
os.makedirs("output", exist_ok=True)
df_final.to_csv("output/nyc_eVTOL_demand_by_borough.csv", index=False)

print("✅ Integrated Python-based demand dataset created successfully!")
print(df_final.head(5))

✅ Integrated Python-based demand dataset created successfully!
            borough  year  month  num_trips  total_pax  avg_distance  \
1178       Brooklyn  2024      3      43939    29084.0     41.750808   
865   Staten Island  2024      2         57       60.0      7.887892   
101        Brooklyn  2024      6      57039    32932.0     24.590772   
439          Queens  2024      2     250312   336690.0     16.660371   
58    Staten Island  2024      5        172      104.0    114.295463   

       avg_fare  population  median_income    vehicles  jobs_total  \
1178  33.458016     2632097   86578.533766  726.494805      834138   
865   47.838181      488645   98995.589744  224.786325      118172   
101   33.341207     2632097   86578.533766  726.494805      834138   
439   41.439409     2320455   90421.181009  452.859050      653726   
58    57.752778      488645   98995.589744  224.786325      118172   

      airport_enplanements  demand_index  
1178                800000      194108.5